In [2]:
from graph2mat4abn.tools.import_utils import load_config, get_object_from_module
from graph2mat4abn.tools.tools import get_basis_from_structures_paths, get_kwargs, load_model
from graph2mat4abn.tools.scripts_utils import get_model_dataset, init_mace_g2m_model
from graph2mat4abn.tools.script_plots import update_loss_plots, plot_grad_norms
from torch_geometric.data import DataLoader
from graph2mat4abn.tools.scripts_utils import generate_g2m_dataset_from_paths
from graph2mat4abn.tools.notebook_utils import add_dotdot_to_str
from pathlib import Path
from mace.modules import MACE, RealAgnosticResidualInteractionBlock
from graph2mat.models import MatrixMACE
from graph2mat.bindings.e3nn import E3nnGraph2Mat
import torch
import warnings
from graph2mat import BasisTableWithEdges

warnings.filterwarnings("ignore", message="The TorchScript type system doesn't support")
warnings.filterwarnings("ignore", message=".*is not a known matrix type key.*")

from joblib import dump, load
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import sisl

In [8]:
# Load model, generate prediction and save it (so that we don't have to load the model again and again)

model_dir = Path("../results/h_crystalls_8")
model_name = "train_best_model.tar"
savedir = Path("./hoppings_vs_distance")

In [5]:
config = load_config(model_dir / "config.yaml")

# Basis generation (needed to initialize the model)
train_paths, val_paths = get_model_dataset(model_dir, verbose=False)

# We have to add "../" to the relative paths because we are in notebooks/
train_paths = [Path(add_dotdot_to_str(p)) for p in train_paths]
val_paths = [Path(add_dotdot_to_str(p)) for p in val_paths]

paths = train_paths + val_paths
basis = get_basis_from_structures_paths(paths, verbose=True, num_unique_z=config["dataset"].get("num_unique_z", None))
table = BasisTableWithEdges(basis)

print("Initializing model...")
model, optimizer, lr_scheduler, loss_fn = init_mace_g2m_model(config, table)

# Load the model
model_path = model_dir / model_name
model, checkpoint, optimizer, lr_scheduler = load_model(model, optimizer, model_path, lr_scheduler=None, initial_lr=None, device='cpu')
history = checkpoint["history"]
print(f"Loaded model in epoch {checkpoint["epoch"]} with training loss {checkpoint["train_loss"]} and validation loss {checkpoint["val_loss"]}.")

Basis computation.
Number of structures to look on: 582
Looking for unique atoms in each structure...


1it [00:00, 72.23it/s]

Found enough basis points. Breaking the search...
Found enough basis points. Breaking the search...
Found the following atomic numbers: [5, 7]
Corresponding path indices: [0, 0]
Basis with 2 elements built!

Basis for atom 0.
	Atom type: 5
	Basis: ((2, 0, 1), (2, 1, -1), (1, 2, 1))
	Basis convention: siesta_spherical
	R: [3.02420918 2.02341372 3.73961942 3.73961942 3.73961942 2.51253945
 2.51253945 2.51253945 3.73961942 3.73961942 3.73961942 3.73961942
 3.73961942]

Basis for atom 1.
	Atom type: 7
	Basis: ((2, 0, 1), (2, 1, -1), (1, 2, 1))
	Basis convention: siesta_spherical
	R: [2.25704422 1.4271749  2.78012609 2.78012609 2.78012609 1.75309697
 1.75309697 1.75309697 2.78012609 2.78012609 2.78012609 2.78012609
 2.78012609]
Initializing model...



/home/ICN2/alapena/miniconda3/envs/graph2mat_upt/lib/python3.12/site-packages/mace/modules/blocks.py:187: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(atomic_energies, dtype=torch.get_default_dtype()),


Using Optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0
)
LR Scheduler:  ReduceLROnPlateau
Arguments:  None
Keyword arguments:  {'cooldown': 0, 'eps': 0.0, 'factor': 0.9, 'min_lr': 1e-09, 'mode': 'min', 'patience': 100}
LOSS FN SELECTED:  block_type_mse
Using Loss function <class 'graph2mat.core.data.metrics.block_type_mse'>
Loaded model in epoch 110870 with training loss 350.75750732421875 and validation loss 39898.26953125.


In [22]:
paths = [
    Path('../dataset/SHARE_OUTPUTS_8_ATOMS/11ad-ba95-4a26-8f92-5267f5787553'), # Cubic. Train of val?
]

for i, path in enumerate(paths):
    
    # Inference
    dataset, processor = generate_g2m_dataset_from_paths(config, basis, table, [path], verbose=False)
    dataloader = DataLoader(dataset, 1)
    model.eval()

    data = next(iter(dataloader))

    with torch.no_grad():
        model_predictions = model(data=data)

        h_pred = processor.matrix_from_data(data, predictions={"node_labels": model_predictions["node_labels"], "edge_labels": model_predictions["edge_labels"]})[0]
        h_true = processor.matrix_from_data(data)[0]

    # Save results
    n_atoms = path.parent.name.split("_")[-2]
    savedir_result = savedir / f"inference_{n_atoms}atm_{path.name}"
    savedir_result.mkdir(parents=True, exist_ok=True)

    np.save(savedir_result / "h_true.npy", h_true)
    np.save(savedir_result / "h_pred.npy", h_pred)

1it [00:00, 14.75it/s]
/home/ICN2/alapena/miniconda3/envs/graph2mat_upt/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Keeping all the dataset in memory.


In [ ]:
# Load the inference
name = "8atm_11ad-ba95-4a26-8f92-5267f5787553"

loaddir = Path(f"./hoppings_vs_distance/inference_{name}")
h_true = np.load(loaddir / "h_true.npy", allow_pickle=True)
h_pred = np.load(loaddir / "h_pred.npy", allow_pickle=True)

In [ ]:
# TODO: plot hoppings vs distance